In [1]:
import numpy as np
import pandas as pd
import scipy

In [2]:
from ecoevocrm.consumer_resource_system import *
from ecoevocrm.consumer_resource_system import *
import ecoevocrm.utils as utils
import ecoevocrm.viz as viz
import ecoevocrm.coarse_graining as cg
import ecoevocrm.strain_pool

In [3]:
from ecoevocrm.landscapes import *

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

----

## Landscapes

In [5]:
N = 3
K = 2

In [6]:
landscape_A = NKLandscape(N=N, K=K, seed=3)
landscape_A.genotype_fitnesses

{'000': 0.5101963174707672,
 '001': 0.16548022933086073,
 '010': 0.3454211491399835,
 '011': 0.27137315120415023,
 '100': 0.586614147551605,
 '101': 0.7139450963169528,
 '110': 0.48191751643299235,
 '111': 0.4957310395484951}

In [7]:
landscape_B = NKLandscape(N=N, K=K, seed=103)
landscape_B.genotype_fitnesses

{'000': 0.586179439191389,
 '001': 0.21702730929748862,
 '010': 0.6479737011374213,
 '011': 0.3379815499842387,
 '100': 0.8198417276563889,
 '101': 0.4840669724385309,
 '110': 0.47810084561968175,
 '111': 0.7045535617163697}

----

## Model parameters

### Traits

Set the number of resources in the system:

In [ ]:
num_resources = 10

Define type(s) that will be present at the start of the simulation:

Here we define the initial population as consisting of only one type (which is the first type from the list of all possible types)

In [ ]:
sigma_allpossible = utils.binary_combinations(num_resources, exclude_all_zeros=True)
sigma_allpossible = sigma_allpossible/sigma_allpossible.sum(axis=1, keepdims=1)

In [ ]:
sigma = sigma_allpossible[0]

The initial composition of the population, as encoded in the $\sigma$ matrix, looks like this:

In [ ]:
viz.matrix_plot(sigma, linecolor='lightgray', linewidths=1)

Set the initial abundance of each type:
    
(Here we set the initial abundance of each type to 1 individual)

In [ ]:
N_init = np.ones(np.atleast_2d(sigma).shape[0])
N_init

### Costs

##### Define cost parameters for types:

Set the baseline cost $\xi$:

In [ ]:
xi = 0.1

Set the cost per trait, $\chi_{i}$:

In [ ]:
chi = 0.3

Set the trait interaction costs by defining a matrix, $\\J_{ij}$, that encodes the cost for each pair of traits:

In [ ]:
# J = None
J = utils.random_matrix((num_resources, num_resources), 'tikhonov_sigmoid', args={'n_star': 5, 'delta': 1}, triangular=True, diagonal=0, 
                        seed=2)
viz.matrix_plot(J, vmin=-0.4, vmax=0.4)

### Environment

Set the initial amount of each resource:

In [ ]:
R_init = np.ones(num_resources)
viz.matrix_plot(R_init, vmin=0, vmax=1, cbar=False, linecolor='lightgray', linewidths=1)

Set the influx of each resource:

(Here the last 5 resources have influx, while the first 5 resources have zero influx)

In [ ]:
rho = np.ones(num_resources)
viz.matrix_plot(rho, vmin=0, vmax=1, cbar=False, linecolor='lightgray', linewidths=1)

Set the time constant for resource dynamics (will be 1 in practically all cases)

In [ ]:
tau = 1

Set the resource dynamics mode used in the simulation. 

(In many basic scenarios we use 'fast resource equilibration' ('fasteq') dynamics)

In [ ]:
resource_dynamics_mode='fasteq'

## Simulate model

In [ ]:
system = ConsumerResourceSystem(type_set=TypeSet(sigma=sigma, xi=xi, chi=chi, J=J, binarize_traits_J_cost_terms=True, mu=1e-10), 
                                rho=rho, tau=tau,
                                N_init=N_init, R_init=R_init, 
                                resource_dynamics_mode='explicit', 
                                max_time_step=1e3)

In [ ]:
T_total = 1e6

In [ ]:
system.run(T=T_total)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
viz.stacked_abundance_plot(system, ax=ax, relative_abundance=True, apply_palette_depth=1, log_x_axis=True, color_seed=1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
viz.resource_plot(system, ax=ax, stacked=False, relative=False, log_x_axis=True, log_y_axis=False)

In [ ]:
viz.matrix_plot(system.extant_type_set.sigma)

---------
---------